In [60]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import os
import pandas as pd
import datetime
import urllib

In [14]:
movie_themes = ['/moviethemes-0','/moviethemes-a','/moviethemes-b','/moviethemes-c','/moviethemes-d','/moviethemes-e','/moviethemes-f','/moviethemes-g','/moviethemes-h','/moviethemes-i','/moviethemes-j','/moviethemes-k','/moviethemes-l','/moviethemes-m','/moviethemes-n','/moviethemes-o','/moviethemes-p','/moviethemes-q','/moviethemes-r','/moviethemes-s','/moviethemes-t','/moviethemes-u','/moviethemes-v','/moviethemes-w','/moviethemes-x','/moviethemes-y'']

In [54]:
links = []
for theme in movie_themes:
    browser = webdriver.Firefox(executable_path=os.getcwd()+'/geckodriver')
    browser.get('https://freemidi.org' + str(theme))
    filmes = browser.find_elements_by_class_name("genre-band-container")
    for filme in filmes:
        url = filme.find_element_by_css_selector('a').get_attribute('href')
        url_download = url[url.find('download2')+10:url.find('download2')+10+(url[url.find('download2')+10:]).find('-')]
        titulo = filme.text.replace(' ','')
        links.append([titulo,url_download])
    browser.close()

In [57]:
links

[['10', '823'],
 ['101Dalmations', '831'],
 ['12Monkeys', '824'],
 ['1492ConquestofParadise', '825'],
 ['1941', '826'],
 ['2Fast2Furious', '11575'],
 ['2001ASpaceOddysey', '827'],
 ['2001:TheYearWeMakeContact', '833'],
 ['2010:TheYearWeMakeContact', '11577'],
 ['4WeddingsAndAFuneral', '828'],
 ['40-YearOldVirgin', '11576'],
 ['48Hours', '832'],
 ['54', '829'],
 ['633Squadron', '2188'],
 ['9AndHalfWeeks', '834'],
 ['9To5', '830'],
 ['ALeagueOfTheirOwn', '1341'],
 ['ANightAtTheRoxberry-WhatIsLove', '1340'],
 ['Abyss', '1355'],
 ['AccidentalTourist', '1356'],
 ['AceVentura-WhenNatureCalls', '1357'],
 ['AceVenturaPetDetective', '1342'],
 ['AddamsFamily', '1343'],
 ['AddictedtoLove', '2189'],
 ['AdvancetotheRear', '2190'],
 ['AdventuresOfBuckarooBanzai', '1358'],
 ['AdventuresOfFordFairlane', '1359'],
 ['AgainstAllOdds', '1360'],
 ['AirForceOne', '1344'],
 ['Aladdin', '1361'],
 ['Aladdin(AFriendLikeMe)', '2191'],
 ['Aladdin(ArabianNights)', '2192'],
 ['Aladdin(OneJump)', '2193'],
 ['Aladdin

In [63]:
errors = []
for link in links:
    try:
        urllib.request.urlretrieve ("https://freemidi.org/getter-"+link[1], 'midi_songs/'+link[0]+'.mid')
    except:
        errors.append(link)

In [66]:
df = pd.read_table('data.tsv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
df_1 = df[["primaryTitle","genres"]]

In [75]:
df_1["primaryTitle"] = df_1["primaryTitle"].apply(lambda x: str(x).replace(' ',''))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
df_movies = df_1.drop_duplicates(subset ="primaryTitle", keep = 'first')

In [176]:
output = pd.DataFrame(data={'primaryTitle':[], 'path':[]})
for file in os.listdir('midi_songs'):
    if not '.DS' in file:
        temp = pd.DataFrame(data={'primaryTitle':[str(file[:file.find('.')])], 'path':[file]})
        output = output.append(temp)

In [179]:
df_songs = output.reset_index(drop=True)

In [180]:
movies_classification = pd.merge(df_songs, df_movies, how='left', on=["primaryTitle"])

In [197]:
df_songs_sort = df_songs.sort_values(by=["primaryTitle"]).reset_index(drop=True)
df_movies_sort = df_movies.sort_values(by=["primaryTitle"]).reset_index(drop=True)

In [198]:
df_songs_sort = df_songs_sort[~pd.isnull(df_songs_sort["primaryTitle"])]
df_movies_sort = df_movies_sort[~pd.isnull(df_movies_sort["primaryTitle"])]

In [199]:
df_songs_sort["primaryTitle"] = df_songs_sort["primaryTitle"].astype(str)
df_movies_sort["primaryTitle"] = df_movies_sort["primaryTitle"].astype(str)

In [203]:
pd.merge_asof(df_songs_sort, df_movies_sort, left_on="primaryTitle", right_on="primaryTitle")

TypeError: 'NoneType' object is not callable

In [164]:
df_songs_2 = df_songs.copy()

In [165]:
df_movies_2 = df_movies.copy()

In [104]:
movies_classification.to_csv('movies_classification.csv')